# CHAID

The EPC data contains several categorical variables with a lot of values. In order to find suitable features which will retain the most information, three feature sets are explored;
* data driven
* domain driven
* exhaustive

The first approach, termed data driven, uses statistical methods to reduce the number of variables. As the variables containing textual descriptions of the property have been created free-hand, many contain a large number of unique values. In some cases, only recorded for one property. The data driven approach uses a single level Chi-square Automatic Interaction Detector (CHAID) to group the levels within each categorical variable into a smaller number of groups. CHAID groups values with a similar response rate or in this context Energy Efficiency Rating (EER). 

This script run CHAID and stores the results in a dictionary

In [2]:
import numpy as np
import pandas as pd
import datetime
import os
import glob
import json
from CHAID import Tree
import re

In [3]:
# set variables from config file
config_path = os.path.abspath('..')

with open(config_path + '/config-example.json', 'r') as f:
    config = json.load(f)

processing_path = config['DEFAULT']['processing_path']
epc_train_fname = config['DEFAULT']['epc_train_fname']
epc_test_fname = config['DEFAULT']['epc_test_fname']
epc_train_clean_fname = config['DEFAULT']['epc_train_clean_fname']
epc_chaid_fname = config['DEFAULT']['epc_chaid_fname']
epc_fname_suffix = config['DEFAULT']['epc_fname_suffix']

In [4]:
dtype_dict = {'INSPECTION_DATE':'str'}

epc_train = pd.read_csv(os.path.join(processing_path,epc_train_clean_fname + epc_fname_suffix),
                        header = 0,
                        delimiter = ',',
                        dtype = dtype_dict,
                        parse_dates = ['INSPECTION_DATE'])

C:\Users\KingRemy\AppData\Local\Temp\ipykernel_5940\425336681.py:3: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  epc_train = pd.read_csv(os.path.join(processing_path,epc_train_clean_fname + epc_fname_suffix),


In [5]:
#Get quantile boundafries
quantiles = epc_train['CURRENT_ENERGY_EFFICIENCY'].describe()
print(quantiles['25%'])
print(quantiles['75%'])

56.0
73.0


In [6]:
#Create a new new target field within the training data
min_eff = epc_train['CURRENT_ENERGY_EFFICIENCY'].min()
max_eff = epc_train['CURRENT_ENERGY_EFFICIENCY'].max()
epc_train['eff_flag'] = pd.cut(epc_train['CURRENT_ENERGY_EFFICIENCY'],
                               bins = [min_eff,56.0,73.0,max_eff],
                               labels = ['0','99','1'])

#Drop unwated '99' level of eff_flag and convert to integer
epc_train = epc_train[epc_train['eff_flag'].isin(['0','1'])]
epc_train['eff_flag'] = epc_train['eff_flag'].astype(int)

In [7]:
#Get the {0,1} sample size by taking the smallest class from the 0
#and 1 outcomes
sample_size = epc_train['eff_flag'].value_counts().min()


#Subsample positive and negative samples
neg_eff_flag = epc_train[epc_train['eff_flag'] == 0].sample(sample_size,random_state = 1234,axis = 0)

pos_eff_flag = epc_train[epc_train['eff_flag'] == 1].sample(sample_size,random_state = 1234,axis = 0)

#Concatenate
epc_chaid = pd.concat([neg_eff_flag, pos_eff_flag])
epc_chaid['eff_flag'].value_counts()

#Randomly shuffle epc_CHAID and reset the index
epc_chaid = epc_chaid.sample(frac = 1).reset_index(drop=True)

In [8]:
epc_chaid

,LMK_KEY,region,POSTCODE,BUILDING_REFERENCE_NUMBER,CURRENT_ENERGY_RATING,CURRENT_ENERGY_EFFICIENCY,PROPERTY_TYPE,BUILT_FORM,INSPECTION_DATE,COUNTY,...,FLOOR_HEIGHT,MECHANICAL_VENTILATION,inspection_year,floors_average_thermal_transmittance,SECONDHEAT_DESCRIPTION,LIGHTING_DESCRIPTION,low_energy_lighting_perc,roof_average_thermal_transmittance,walls_average_thermal_transmittance,eff_flag
0,76804109132019012910583757268502,Arun,BN16 3QD,267555468,D,55,Bungalow,Semi-Detached,2019-01-28,West Sussex,...,NaN,natural,2019,NaN,"Room heaters, dual fuel (mineral and wood)",low energy lighting 20% of fixed outlets,20.0,NaN,NaN,0
1,437757832032010021611513749968304,Elmbridge,KT10 0DD,3419362768,C,80,House,Semi-Detached,2010-02-15,Surrey,...,2.6,NaN,2010,0.2,None,low energy lighting 20% of fixed outlets,20.0,0.2,0.3,1
2,ccade78ea55a3622878cf9df2a8466a03ca1d7c81b7537...,Crawley,RH10 1LH,10000576650,B,83,Flat,Mid-Terrace,2020-09-23,West Sussex,...,2.4,NaN,2020,NaN,None,low energy lighting in all fixed outlets,NaN,0.1,0.2,1
3,1007597359222013091408274323838177,Babergh,CO10 5QA,7266583178,C,75,House,Mid-Terrace,2013-08-27,Suffolk,...,NaN,natural,2013,NaN,None,low energy lighting in all fixed outlets,NaN,NaN,NaN,1
4,1624693839262018041915261627888823,Hertsmere,WD23 2NW,2227357578,B,82,Flat,NaN,2017-09-28,Hertfordshire,...,NaN,NaN,2017,0.1,None,low energy lighting in all fixed outlets,NaN,NaN,0.2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1912829,1239696939922014112111575550718294,Chelmsford,CM2 0AJ,9529320378,B,84,Flat,Semi-Detached,2014-11-19,Essex,...,NaN,natural,2014,NaN,None,low energy lighting in all fixed outlets,NaN,NaN,NaN,1
1912830,1233202279022014110709223689738684,Norwich,NR5 8EP,8862779278,B,84,Flat,Semi-Detached,2014-11-07,Norfolk,...,NaN,NaN,2014,NaN,None,low energy lighting in all fixed outlets,NaN,NaN,0.2,1
1912831,1445479299922016051913113304238496,Lancaster,LA3 3AH,9057974478,F,25,Bungalow,Semi-Detached,2016-05-17,Lancashire,...,2.8,natural,2016,NaN,"Room heaters, mains gas",low energy lighting 90% of fixed outlets,90.0,NaN,NaN,0
1912832,350210149452009082116031204210661,Braintree,CO9 2BA,6167946668,F,38,House,Semi-Detached,2009-08-21,Essex,...,2.6,natural,2009,NaN,"Room heaters, coal",low energy lighting 20% of fixed outlets,20.0,NaN,NaN,0


In [9]:
#Numeric
var_list_num = epc_chaid.select_dtypes(include= 'number').columns.tolist()
var_list_num.remove('CURRENT_ENERGY_EFFICIENCY')

#Categorical
var_list_cat = epc_chaid.select_dtypes(include= ['object','category']).columns.tolist()
var_list_cat.remove('LMK_KEY')
# var_list_cat.remove('POSTCODE')
var_list_cat.remove('CURRENT_ENERGY_RATING')

### Creating a dictionary of CHAID scores

In [10]:
chaid_dict = {}
for var in var_list_cat:
    #Set the inputs and outputs
    #The imputs are given as a dictionary along with the type
    #The output must be of string type
    #I have assume all features are nominal, we can change the features dictionary to include the ordinal type
    features = {var:'nominal'}
    label = 'eff_flag'
    #Create the Tree
    chaid_dict[var] = {}
    tree = Tree.from_pandas_df(epc_chaid, i_variables = features, d_variable = label, alpha_merge = 0.0)
    #Loop through all the nodes and enter into a dictionary
    print(f'\n\n\nVariable: {var}')
    print(f'p-value: {tree.tree_store[0].split.p}')
    print(f'Chi2: {tree.tree_store[0].split.score}')
    for i in range(1, len(tree.tree_store)):
        count = tree.tree_store[i].members[0] + tree.tree_store[i].members[1]
        rate = tree.tree_store[i].members[1] / count
        print(f'\nNode {i}:\n\tCount = {count}\tRate = {rate}')
        print(f'\t{tree.tree_store[i].choices}')
        chaid_dict[var]['node' + str(i)] = tree.tree_store[i].choices




Variable: region
p-value: 0.0
Chi2: 65517.82722484344

Node 1:
	Count = 433464.0	Rate = 0.45513352896665005
	['Adur', 'Gedling', 'Bassetlaw', 'Cotswold', 'Teignbridge', 'Boston', 'Thanet', 'Hastings', 'Ashfield', 'Babergh', 'Lewes', 'Waverley', 'Breckland', 'Guildford', 'Worcester', 'Fylde', 'Wealden', 'Bolsover', 'Brentwood', 'Carlisle', 'Broadland', 'Rochford', 'Dover', 'Stroud', 'Chichester']

Node 2:
	Count = 301953.0	Rate = 0.37460796878984476
	['Allerdale', 'Rossendale', 'Craven', 'Broxtowe', 'Ryedale', 'Torridge', 'Copeland', 'Richmondshire', 'Erewash', 'Tendring', 'Worthing', 'Rother', 'Scarborough', 'Hambleton', 'Harrogate', 'Lancaster', 'Wyre', 'Melton', 'Sevenoaks', 'Maldon']

Node 3:
	Count = 364964.0	Rate = 0.5059348319286285
	['Arun', 'Blaby', 'Sedgemoor', 'Braintree', 'Swale', 'Havant', 'Stafford', 'Canterbury', 'Cheltenham', 'Preston', 'Bromsgrove', 'Spelthorne', 'Chesterfield', 'Fenland', 'Mansfield', 'Elmbridge', 'Mendip', 'Rushcliffe', 'Tandridge', 'Gravesham']

N

In [ ]:
%store chaid_dict

Stored 'chaid_dict' (dict)
